# Bibliotecas

In [6]:
import pandas as pd
from pathlib import Path
import os

# Tratar bases

In [34]:
# !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
# SÓ RODAR CASO AS BASES NÃO ESTIVEREM TRATADAS
# !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

# Diretório base onde os arquivos CSV estão localizados
base_diretório = Path(os.getcwd())
diretório_princapl = base_diretório.parent
caminho = diretório_princapl / 'Bases'

# Definir os caminhos para cada criptomoeda
arquivos = {
    'Bitcoin': caminho / 'Bitcoin Historical Data.csv',
    'Ethereum': caminho / 'Ethereum Historical Data.csv',
    'BNB': caminho / 'BNB Historical Data.csv',
    'Solana': caminho / 'Solana Historical Data.csv'
}

# Ler o arquivo ano.txt e pegar o ano
with open(diretório_princapl/'data'/'ano.txt', 'r') as file:
    ano = int(file.read().strip())

# Função para tratar os dados de cada criptomoeda
def tratar_dados_cripto(caminho_csv):
    # Ler o arquivo CSV
    acao = pd.read_csv(caminho_csv)

    # Formatar a coluna de data
    acao['Date'] = pd.to_datetime(acao['Date'], format='%m/%d/%Y')

    # Converter as colunas de dinheiro para float
    colunas_dinheiro = ['Price', 'Open', 'High', 'Low']
    for coluna in colunas_dinheiro:
        if acao[coluna].dtype == 'object':  
            acao[coluna] = pd.to_numeric(acao[coluna].str.replace(',', ''))

    # Converter a coluna 'Change %' para float
    if acao['Change %'].dtype == 'object':  
        acao['Change %'] = pd.to_numeric(acao['Change %'].str.replace('%', ''))

    # Função para converter o volume (coluna 'Vol.') para número
    def converter_vol_para_numero(valor):
        if isinstance(valor, str):  # Verifica se o valor é uma string
            if 'K' in valor:
                return float(valor.replace('K', '').replace(',', '')) * 1000
            elif 'M' in valor:
                return float(valor.replace('M', '').replace(',', '')) * 1000000
            elif 'B' in valor:
                return float(valor.replace('B', '').replace(',', '')) * 1000000000
            else:
                return float(valor.replace(',', ''))
        else:
            return valor  # Retorna o valor diretamente se já for numérico

    # Aplicar a função na coluna 'Vol.'
    acao['Vol.'] = acao['Vol.'].apply(converter_vol_para_numero)

    # Ordenar os dados pela data
    acao = acao.sort_values(by='Date', ascending=True)

    # Renomear a coluna 'Vol.' para 'Volume'
    acao = acao.rename(columns={'Vol.': 'Volume'})

    return acao

# Aplicar a função para cada criptomoeda e armazenar o resultado em um DataFrame
df_bitcoin = tratar_dados_cripto(arquivos['Bitcoin'])
df_ethereum = tratar_dados_cripto(arquivos['Ethereum'])
df_bnb = tratar_dados_cripto(arquivos['BNB'])
df_solana = tratar_dados_cripto(arquivos['Solana'])

df_bitcoin.to_csv("Bitcoin Historical Data.csv", index=False)
df_ethereum.to_csv("Ethereum Historical Data.csv", index=False)
df_bnb.to_csv("BNB Historical Data.csv", index=False)
df_solana.to_csv("Solana Historical Data.csv", index=False)
